In [30]:
# vocabulary: abc def -
#             123 456 0
# sample n=6 m=2
# bd- dee
# dee ffa
vocabulary = "-abcdef"

In [35]:
# CTC matrix
import numpy as np

predictions = np.array([[2, 0, 0, 4, 5, 5],
                        [4, 5, 5, 6, 6, 1]])
print(predictions.shape)
indices = predictions.flatten()
print(indices)

(2, 6)
[2 0 0 4 5 5 4 5 5 6 6 1]


In [28]:
# generate adjacencies
def connect(predictions, t, k, n, predictions_shape):
    edges = []
    for j in range(predictions_shape):  # offset
        if predictions[j][k] != 0:
            edges.append([n*predictions.shape[1]+t, j*predictions.shape[1]+k])
        else:
            # skip to next if exists
            edges.extend(connect(predictions, t, k+1, n, predictions_shape))
    return edges

edges = []
for t in range(predictions.shape[1]-1):  # columns
    for n in range(predictions.shape[0]):  # rows
        if predictions[n][t] != 0:
            edges.extend(connect(predictions, t, t+1, n, predictions.shape[0]))
            
print(edges)

[[0, 3], [0, 9], [0, 8], [0, 7], [6, 3], [6, 9], [6, 8], [6, 7], [7, 3], [7, 9], [7, 8], [8, 3], [8, 9], [3, 4], [3, 10], [9, 4], [9, 10], [4, 5], [4, 11], [10, 5], [10, 11]]


In [44]:
# search
query_str = 'bed'
query = [vocabulary.index(letter) for letter in query_str]
query = [query[i:i + 2] for i in range(0, len(query)-1, 1)]
query

[[2, 5], [5, 4]]

In [41]:
bigrams = []
for e in edges:
    bigram = [indices[e[0]], indices[e[1]]]
    if bigram in query:
        bigrams.append(e)
bigrams

[[0, 8], [0, 7], [7, 3], [8, 3]]

In [45]:
import networkx as nx

DG = nx.DiGraph()

DG.add_edges_from(bigrams)
path = nx.dag_longest_path(DG)
print(path)
word = ''.join([vocabulary[indices[i]] for i in path])
print(word)
word == query_str

[0, 7, 3]
bed


True